This notebook generates the following panels:
- ( ) Summary: distribution of depth preferences of all cells

use env 2p_analysis_cottage

In [ ]:
VERSION = 1

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import defopt
import pickle
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import curve_fit
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

mpl.rcParams["pdf.fonttype"] = 42  # save text as text not outlines
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap

# from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings

import flexiznam as flz

from cottage_analysis.depth_analysis.filepath.generate_filepaths import *
from cottage_analysis.imaging.common import find_frames
from cottage_analysis.imaging.common import imaging_loggers_formatting as format_loggers
from cottage_analysis.stimulus_structure import sphere_structure as vis_stim_structure
from cottage_analysis.depth_analysis.plotting.plotting_utils import *
from cottage_analysis.depth_analysis.plotting.basic_vis_plots import *
from cottage_analysis.depth_analysis.depth_preprocess.process_params import *
from cottage_analysis.depth_analysis.depth_preprocess.process_trace import *

In [ ]:
save_root = (
    "/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver"
    + str(VERSION)
    + "/preferred_depth_distribution/"
)
if not os.path.exists(save_root):
    os.makedirs(save_root)

rawdata_root = "/camp/lab/znamenskiyp/data/instruments/raw_data/projects/"
root = "/camp/lab/znamenskiyp/home/shared/projects/"
project = "hey2_3d-vision_foodres_20220101"
protocol = "SpheresPermTubeReward"
V1_sessions = [
    [
        "S20220512",
        "S20220511",
        "S20220429",
        "S20220419",
        "S20220426",
        "S20220506",
        "S20220503",
        "S20220505",
        "S20220516",
    ],
    [
        "S20220504",
        "S20220426",
        "S20220421",
        "S20220419",
        "S20220505",
        "S20220503",
        "S20220510",
        "S20220512",
        "S20220511",
    ],
]
summary_dir = root + "/" + project + "/Analysis/summary/"
# V1 150: ['S20220512','S20220511','S20220429']['S20220504','S20220426','S20220421']
# V1 200: ['S20220419','S20220426','S20220506']['S20220419','S20220505','S20220503']
# V1 250: ['S20220503','S20220505','S20220516']['S20220510','S20220512','S20220511']

mouses = ["PZAH6.4b", "PZAG3.4f"]
depth_list = [0.06, 0.19, 0.6, 1.9, 6]
depth_thr_min = 0.019
depth_thr_max = 19

all_results = pd.DataFrame(
    columns=[
        "ROI",
        "preferred_depth_idx",
        "log_amplitude",
        "xo_logged",
        "yo_logged",
        "log_sigma_x2",
        "log_sigma_y2",
        "theta",
        "offset",
        "r_sq",
        "preferred_depth_max",
        "preferred_depth_gaussian",
    ]
)
for k in range(len(V1_sessions)):
    mouse = mouses[k]
    for session in V1_sessions[k]:

        # ----- STEP1: Generate file path -----
        print("---START STEP 1---", "\n", "Getting data filepaths...", flush=True)
        (
            rawdata_folder,
            protocol_folder,
            analysis_folder,
            suite2p_folder,
            trace_folder,
        ) = generate_filepaths.generate_file_folders(
            project=project,
            mouse=mouse,
            session=session,
            protocol=protocol,
            rawdata_root=rawdata_root,
            root=root,
        )
        #             print(rawdata_folder, protocol_folder, analysis_folder, suite2p_folder, trace_folder, flush=True)

        filename = analysis_folder / (
            "plane0/gaussian_blob_fit_new_" + str(0.1) + ".pickle"
        )
        filename2 = analysis_folder / (
            "plane0/gaussian_depth_tuning_fit_new_" + str(0.5) + ".pickle"
        )
        with open(filename, "rb") as handle:
            results_blob = pickle.load(handle)
        with open(filename2, "rb") as handle:
            results_depth = pickle.load(handle)

        # Load suite2p files
        ops = np.load(suite2p_folder / "ops.npy", allow_pickle=True)
        ops = ops.item()
        iscell = np.load(suite2p_folder / "iscell.npy", allow_pickle=True)[:, 0]
        F = np.load(trace_folder / "F.npy", allow_pickle=True)
        Fast = np.load(trace_folder / "Fast.npy", allow_pickle=True)

        # All_rois
        which_rois = (np.arange(F.shape[0]))[iscell.astype("bool")]

        # Load depth neurons
        depth_neurons = np.load(analysis_folder / ("plane0/depth_neurons.npy"))
        max_depths = np.load(analysis_folder / ("plane0/max_depths_index.npy"))

        # Append info to all_results
        results_blob["preferred_depth_gaussian"] = np.exp(
            results_depth["x0_logged"].astype("float64")
        )
        results_blob["preferred_depth_max"] = [
            depth_list[int(max_depth)] * 100
            for max_depth in results_blob["preferred_depth_idx"]
        ]
        all_results = pd.concat([all_results, results_blob], ignore_index=True)
        print(session)

with open(save_root + "gaussian_fit_summary_old2.pickle", "wb") as handle:
    pickle.dump(all_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(
    "/camp/lab/znamenskiyp/home/shared/projects/hey2_3d-vision_foodres_20220101/Analysis/PZAH8.2i/S20230220/SpheresPermTubeReward/plane0/gaussian_blob_fit_new_0.1_actual_running.pickle",
    "rb",
) as handle:
    results_blob = pickle.load(handle)
results_blob

In [ ]:
save_root = (
    "/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver"
    + str(VERSION)
    + "/preferred_depth_distribution/"
)
if not os.path.exists(save_root):
    os.makedirs(save_root)

rawdata_root = "/camp/lab/znamenskiyp/data/instruments/raw_data/projects/"
root = "/camp/lab/znamenskiyp/home/shared/projects/"
project = "hey2_3d-vision_foodres_20220101"
protocol = "SpheresPermTubeReward"
V1_sessions = [
    ["S20230116", "S20230127", "S20230202"],
    ["S20230116", "S20230117", "S20230203", "S20230209", "S20230216", "S20230220"],
    [
        "S20230117",
        "S20230126",
        "S20230131",
        "S20230202",
        "S20230206",
        "S20230210",
        "S20230213",
        "S20230214",
    ],
]
summary_dir = root + "/" + project + "/Analysis/summary/"

mouses = ["PZAH8.2h", "PZAH8.2i", "PZAH8.2f"]
depth_list = [0.05, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4]

columns = [
    "ROI",
    "preferred_depth_idx",
    "log_amplitude",
    "xo_logged",
    "yo_logged",
    "log_sigma_x2",
    "log_sigma_y2",
    "theta",
    "offset",
    "r_sq",
    "preferred_depth_max",
    "preferred_depth_gaussian",
]
all_results = pd.DataFrame(columns=columns)
for k in range(len(V1_sessions)):
    mouse = mouses[k]
    for session in V1_sessions[k]:

        # ----- STEP1: Generate file path -----
        print("---START STEP 1---", "\n", "Getting data filepaths...", flush=True)
        flexilims_session = flz.get_flexilims_session(project_id=project)
        sess_children = get_session_children(
            project=project,
            mouse=mouse,
            session=session,
            flexilims_session=flexilims_session,
        )
        root = Path(flz.PARAMETERS["data_root"]["processed"])

        # ----- STEP1: Generate file path -----
        session_analysis_folder_original = (
            root / project / "Analysis" / mouse / session / protocol
        )
        session_protocol_folder = root / project / mouse / session / protocol
        session_analysis_folder = (
            root / project / "Analysis" / mouse / session / protocol
        )
        (_, _, _, suite2p_folder, _,) = generate_file_folders(
            project=project,
            mouse=mouse,
            session=session,
            protocol=protocol,
            all_protocol_recording_entries=None,
            recording_no=0,
            flexilims_session=None,
        )

        filename = (
            session_analysis_folder
            / "plane0/gaussian_blob_fit_new_0.1_actual_running.pickle"
        )

        with open(filename, "rb") as handle:
            results_blob = pickle.load(handle)

        # Load depth neurons
        depth_neurons = np.load(session_analysis_folder / ("plane0/depth_neurons.npy"))
        max_depths = np.load(session_analysis_folder / ("plane0/max_depths_index.npy"))

        # Append info to all_results
        results_blob["preferred_depth_max"] = [
            depth_list[int(max_depth)] * 100
            for max_depth in results_blob["preferred_depth_idx"]
        ]
        results_blob = results_blob[columns]
        all_results = pd.concat([all_results, results_blob], ignore_index=True)
        print(session)

with open(save_root + "gaussian_fit_summary_new3.pickle", "wb") as handle:
    pickle.dump(all_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(save_root + "gaussian_fit_summary_old2.pickle", "rb") as handle:
    all_results_old2 = pickle.load(handle)
with open(save_root + "gaussian_fit_summary_new3.pickle", "rb") as handle:
    all_results_new3 = pickle.load(handle)
all_results = pd.concat([all_results_old2, all_results_new3], ignore_index=True)
with open(save_root + "gaussian_fit_summary_5.pickle", "wb") as handle:
    pickle.dump(all_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
save_root = (
    "/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver"
    + str(VERSION)
    + "/preferred_depth_distribution/"
)
with open(save_root + "gaussian_fit_summary_5.pickle", "rb") as handle:
    all_results = pickle.load(handle)

fontsize_dict = {
    "xlabel": 20,
    "ylabel": 20,
    "xticks": 20,
    "yticks": 20,
    "title": 20,
    "legend": 10,
}

plt.figure(figsize=(5, 5))
depth_min = 1.9
depth_max = 1900
bin_num = 50
depth_bins = np.logspace(np.log(depth_min), np.log(depth_max), num=bin_num, base=np.e)
n, _, _ = plt.hist(
    all_results["preferred_depth_gaussian"],
    bins=depth_bins,
    weights=np.ones(len(all_results)) / len(all_results),
    color="dimgrey",
)
# plt.vlines(x=np.array(depth_list)*100,ymin=0,ymax=3/100,colors='r',linestyles='dashed')
plt.xscale("log")
plt.ylabel("Proportion of neurons", fontsize=fontsize_dict["ylabel"])
plt.xlabel("Preferred depth (cm)", fontsize=fontsize_dict["xlabel"])
plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
despine()
plt.tight_layout(pad=1)
plt.savefig(save_root + "hist_preferred_depth.pdf")